In [4]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from keras.layers import BatchNormalization
from keras.optimizers import SGD

In [5]:
#parameters
batch_size = 32
num_classes = 10
epochs = 10
(x_train, y_train), (x_test, y_test) = mnist.load_data()

print(np.shape(x_train))
print(np.shape(y_train))
print(np.shape(x_test))
print(np.shape(y_test))

#Image dataset dimensions
img_rows, img_cols = 28, 28
print(img_rows, img_cols)

#Reshape the dataset to have a single colour channel
x_train = x_train.reshape((x_train.shape[0], img_rows, img_cols, 1))
x_test = x_test.reshape((x_test.shape[0], img_rows, img_cols, 1))

#One hot encode target values
from keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

#Convert from integers to floats to divide the pixel value from the max value
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# normalize to range 0-1
x_train = x_train / 255.0
x_test = x_test / 255.0

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

#Adding Random Noise
noise_factor = 0.25

x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)
#y_train_noisy = np.clip(x_train_noisy, 0., 1.)
#y_test_noisy = np.clip(x_test_noisy, 0., 1.)


y_train_noisy = y_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=y_train.shape)
y_test_noisy = y_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=y_test.shape)

print(np.shape(x_train_noisy))
print(np.shape(y_train_noisy))
print(np.shape(x_test_noisy))
print(np.shape(y_test_noisy))

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)
28 28
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
(60000, 28, 28, 1)
(60000, 10)
(10000, 28, 28, 1)
(10000, 10)


**Improving Model Accuracy**

Solutions:
  1. Add normalization
  2. Increase Model depth

Adding Normalization

In [6]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
# compile model
opt = SGD(lr=0.01, momentum=0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x_train_noisy, y_train_noisy,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test_noisy, y_test_noisy))

score = model.evaluate(x_test_noisy, y_test_noisy, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print('> %.3f' % (score[1] * 100.0))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 62s 1ms/step - loss: 0.4485 - accuracy: 0.8353 - val_loss: 0.0656 - val_accuracy: 0.9243
Epoch 2/10
60000/60000 [==============================] - 61s 1ms/step - loss: 0.2671 - accuracy: 0.8762 - val_loss: -0.0158 - val_accuracy: 0.9308
Epoch 3/10
60000/60000 [==============================] - 61s 1ms/step - loss: 0.2413 - accuracy: 0.8634 - val_loss: -0.0998 - val_accuracy: 0.9346
Epoch 4/10
60000/60000 [==============================] - 61s 1ms/step - loss: 0.2636 - accuracy: 0.8457 - val_loss: -0.0234 - val_accuracy: 0.9295
Epoch 5/10
60000/60000 [==============================] - 61s 1ms/step - loss: 0.4672 - accuracy: 0.8195 - val_loss: 0.0035 - val_accuracy: 0.9094
Epoch 6/10
60000/60000 [==============================] - 62s 1ms/step - loss: 1.2941 - accuracy: 0.7545 - val_loss: 0.8260 - val_accuracy: 0.8206
Epoch 7/10
60000/60000 [==============================] - 61s 1ms

Increase Model depth

In [8]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Dense(10, activation='softmax'))
# compile model
opt = SGD(lr=0.01, momentum=0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x_train_noisy, y_train_noisy,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test_noisy, y_test_noisy))

score = model.evaluate(x_test_noisy, y_test_noisy, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print('> %.3f' % (score[1] * 100.0))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 112s 2ms/step - loss: 0.2381 - accuracy: 0.9097 - val_loss: 0.0592 - val_accuracy: 0.9321
Epoch 2/10
60000/60000 [==============================] - 110s 2ms/step - loss: 0.1179 - accuracy: 0.9311 - val_loss: -0.0239 - val_accuracy: 0.9308
Epoch 3/10
60000/60000 [==============================] - 111s 2ms/step - loss: 0.0353 - accuracy: 0.9294 - val_loss: -0.1554 - val_accuracy: 0.9410
Epoch 4/10
60000/60000 [==============================] - 110s 2ms/step - loss: -0.0599 - accuracy: 0.9222 - val_loss: -0.0981 - val_accuracy: 0.9368
Epoch 5/10
60000/60000 [==============================] - 114s 2ms/step - loss: -0.1520 - accuracy: 0.9159 - val_loss: 0.7841 - val_accuracy: 0.9229
Epoch 6/10
60000/60000 [==============================] - 110s 2ms/step - loss: 0.0280 - accuracy: 0.9020 - val_loss: -0.0701 - val_accuracy: 0.9213
Epoch 7/10
60000/60000 [==============================] 